## Applying Energy Path algtm to road photos

In [143]:
import cv2
import numpy as np
import matplotlib as plt

In [144]:
RED = (0, 0, 255)

In [145]:
img = cv2.imread("C:\Programs/facultate/licenta/my-autonomous-car/resources/fullcurve.png")


In [146]:
print(img)

[[[207 212 209]
  [220 225 222]
  [224 230 225]
  ...
  [104 109 108]
  [107 111 111]
  [111 116 115]]

 [[201 207 203]
  [224 230 226]
  [222 228 223]
  ...
  [175 180 179]
  [175 180 179]
  [174 179 178]]

 [[181 187 184]
  [227 232 229]
  [224 230 225]
  ...
  [185 189 189]
  [183 188 187]
  [181 186 185]]

 ...

 [[ 66  70  64]
  [ 66  70  64]
  [ 66  70  64]
  ...
  [ 56  60  56]
  [ 56  60  55]
  [ 56  60  55]]

 [[ 66  70  64]
  [ 66  70  64]
  [ 66  70  64]
  ...
  [ 56  60  56]
  [ 56  60  55]
  [ 56  60  55]]

 [[ 66  70  64]
  [ 66  70  64]
  [ 66  70  64]
  ...
  [ 56  60  56]
  [ 56  60  55]
  [ 56  60  55]]]


In [147]:

cv2.imshow("original",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [148]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow("gray",gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Gaussian Blur

In [149]:
blurred = cv2.GaussianBlur(gray,(3,3),3)

cv2.imshow("blurred",blurred)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Bilateral blur


In [150]:
bilateralBlurred = cv2.bilateralFilter(gray, 11, 21, 7)

cv2.imshow("bilateral Blurred",bilateralBlurred)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [151]:
blurred = cv2.GaussianBlur(gray,(3,3),3)
bilateralBlurred = cv2.bilateralFilter(gray, 11, 21, 7)

cv2.imshow("bilateral Blurred",bilateralBlurred)


cv2.imshow("blurred",blurred)

cv2.waitKey(0)
cv2.destroyAllWindows()

### Classic binary thresholding

In [152]:
_, threshClassis = cv2.threshold(blurred, 170, 255, cv2.THRESH_BINARY)
# cv2.imwrite("classicTresh.jpg",threshClassis)
cv2.imshow("classic thresh",threshClassis)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Otsu's thresholding

In [153]:
T, threshOtsu = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# cv2.imwrite("classicTresh.jpg",threshClassis)
print(T)
cv2.imshow("classic thresh",threshOtsu)
cv2.waitKey(0)
cv2.destroyAllWindows()

137.0


### Adaptive binary thresholding

In [154]:
threshAdap = cv2.adaptiveThreshold(blurred, 255,
	cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 5, 10)

cv2.imshow("adaptive thresh",threshAdap)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [155]:
blurred = cv2.GaussianBlur(gray,(5,5),0)
# blurred = cv2.equalizeHist(blurred)

# threshAdap = cv2.adaptiveThreshold(blurred, 255,
#     cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
_, threshClassis = cv2.threshold(blurred, 170, 255, cv2.THRESH_BINARY)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
closed = cv2.morphologyEx(threshClassis, cv2.MORPH_OPEN, kernel)
cv2.imshow("blurred",closed)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Morphological ops

In [156]:
kernel = np.ones((5,5),np.uint8)
closing = cv2.morphologyEx(threshClassis, cv2.MORPH_CLOSE, kernel)
cv2.imshow("closed img",closing)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [157]:
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(threshClassis, cv2.MORPH_OPEN, kernel)
cv2.imshow("opened img",opening)
cv2.waitKey(0)
cv2.destroyAllWindows()

### TESTS

In [158]:
_, threshClassis = cv2.threshold(blurred, 185, 255, cv2.THRESH_BINARY)
# kernel = np.ones((3,3),np.uint8)
kernel = cv2.getGaussianKernel(5,5)
closing = cv2.morphologyEx(threshClassis, cv2.MORPH_CLOSE, kernel)
opening = cv2.morphologyEx(threshClassis, cv2.MORPH_OPEN, kernel)

# cv2.imwrite("classicTresh.jpg",threshClassis)
cv2.imshow("classic thresh",threshClassis)
cv2.imshow("opened img",opening)
cv2.imshow("closed img",closing)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [159]:
blurred = cv2.GaussianBlur(gray,(3,3),0)
cv2.imshow("blur clasic img",blurred)
bilateralBlurred = cv2.bilateralFilter(gray, 11, 21, 7)
cv2.imshow("blur bilateral img",bilateralBlurred)
_, threshClassis = cv2.threshold(blurred, 185, 255, cv2.THRESH_BINARY)
cv2.imshow("thresh clasic  + blurimg",threshClassis)
_, threshClassis = cv2.threshold(bilateralBlurred, 185, 255, cv2.THRESH_BINARY)
cv2.imshow("thresh clasic + biblur img",threshClassis)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
closing = cv2.morphologyEx(threshClassis, cv2.MORPH_CLOSE, kernel)
cv2.imshow("closed img",closing)
kernel1 = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(threshClassis, cv2.MORPH_OPEN, kernel1)
# cv2.imshow("opened img",opening)
kernel2 = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel2)
# cv2.imshow("both img",opening)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Gradient making

In [160]:
def compute_energy(img):
    """
    calculeaza energia la fiecare pixel pe baza gradientului
    :param img: imaginea initiala
    :return:E - energia
    """
    # urmati urmatorii pasi:
    # 1. transformati imagine in grayscale
    # 2. folositi filtru sobel pentru a calcula gradientul in directia X si Y
    # 3. calculati magnitudinea pentru fiecare pixel al imaginii
    E = np.zeros((img.shape[0],img.shape[1]))
    # img_grey = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    grad_x = cv2.Sobel(img,ddepth=cv2.CV_16S,dx=1,dy=0)
    grad_y = cv2.Sobel(img,ddepth=cv2.CV_16S,dx=0,dy=1)

    abs_x = np.abs(grad_x)
    abs_y = np.abs(grad_y)

    E = abs_x + abs_y

    return E

In [161]:
def compute_energy_v2(img):
    ksize = 3

    gX = cv2.Sobel(img, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=ksize)
    gY = cv2.Sobel(img, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=ksize)

    gX = cv2.convertScaleAbs(gX)
    gY = cv2.convertScaleAbs(gY)

    # combined = cv2.addWeighted(gX, 0.5, gY, 0.5, 0)

    magnitude = np.sqrt((gX ** 2) + (gY ** 2))
    # magnitude = gX + gY

    return magnitude


In [162]:
combined = compute_energy_v2(threshClassis)

# cv2.imshow("pre", pre)
# cv2.imshow("Sobel/Scharr Combined", combined)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [163]:
gradient = compute_energy(opening)
cv2.imshow("gradient img",gradient)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [164]:
def select_dynamic_programming_path_going_up(E):
    M = np.zeros(E.shape)
    M[0,:] = E[0,:]
    for i in range(1,E.shape[0]):
        for j  in range (E.shape[1]):
            if j == 0:
                M[i][j] = E[i,j]+max(M[i-1,0],M[i-1,1])
            elif j == E.shape[1] - 1:
                M[i,j] = E[i,j]+max(M[i-1,j],M[i-1,j-1])
            else:
                M[i,j] = E[i,j]+max(M[i-1,j+1],M[i-1,j],M[i-1,j-1])
    line = M.shape[0] - 1
    col = np.argmax(M[line,:])
    # print(np.max(M[line,:]))
    path = [0 for i in range(line+1)]

    path[line] = (line,col)
    
    for line in range(M.shape[0]-2,-1,-1):
        if col == 0:
            if M[line,0] < M[line,1]:
                new_col = 1
            else:
                new_col = 0
        elif col == E.shape[1] - 1:
            if M[line,col] < M[line,col-1]:
                new_col = col - 1
            else:
                new_col = col
        else:
            neigh = np.array([M[line,col-1],M[line,col],M[line,col+1]])
            new_col = col+np.argmax(neigh) - 1

        path[line]= (line,new_col)

        col = new_col

    return path

In [165]:
def select_dynamic_programming_path_going_sideways(E):
    M = np.zeros(E.shape)
    M[0,:] = E[0,:]
    for i in range(1,E.shape[0]):
        for j  in range (E.shape[1]):
            if j == 0:
                M[i][j] = E[i,j]+max(M[i-1,0],M[i-1,1])
            elif j == E.shape[1] - 1:
                M[i,j] = E[i,j]+max(M[i-1,j],M[i-1,j-1])
            else:
                M[i,j] = E[i,j]+max(M[i-1,j+1],M[i-1,j],M[i-1,j-1],M[i][j+1],M[i][j-1])
    line = M.shape[0] - 1
    col = np.argmax(M[line,:])
    # print(np.max(M[line,:]))
    path = [0 for _ in range(60)]
    # path = []
    index = 0

    path[index] = (line,col)

    lineIndex = M.shape[0]-1
    nextLine = False
    while lineIndex > 0 and col >= 0 and col < M.shape[1] and index != 50:
        if col == 0:
            # if M[lineIndex,0] < M[lineIndex,1]:
            #     new_col = 1
            #     nextLine = False
            # else:
            #     new_col = 0
            #     nextLine = False
            if M[lineIndex-1,0] < M[lineIndex-1,1]:
                new_col = 1
                nextLine = True
            else:
                new_col = 0
                nextLine = True
        elif col == E.shape[1] - 1:
            if M[lineIndex-1,col] < M[lineIndex-1,col-1]:
                new_col = col - 1
                nextLine = True
            else:
                new_col = col
                nextLine = True
        else:
            same_line = np.array([M[lineIndex,col-1],M[lineIndex,col+1]])
            upper_line = np.array([M[lineIndex-1,col-1],M[lineIndex-1,col],M[lineIndex-1,col+1]])
            if np.max(same_line) > np.max(upper_line):
                new_col = col+np.argmax(same_line) - 1
                nextLine = False
            else:
                new_col = col+np.argmax(upper_line) - 1
                nextLine = True

        if nextLine:
            lineIndex -= 1

        index += 1  
        path[index]= (lineIndex,new_col)

        col = new_col
    
    return path

In [166]:
def select_dynamic_programming_path_going_up_point_start(E,point): #TODO Sa incerc sa tin pathul in interiorul benzii si sa nu fac pe 2 imagini diferite    
    M = np.zeros(E.shape)
    M[0,:] = E[0,:]
    for i in range(1,E.shape[0]):
        for j  in range (E.shape[1]):
            if j == 0:
                M[i][j] = E[i,j]+max(M[i-1,0],M[i-1,1])
            elif j == E.shape[1] - 1:
                M[i,j] = E[i,j]+max(M[i-1,j],M[i-1,j-1])
            elif j >= 2 and j <= E.shape[1] - 3:
                M[i,j] = E[i,j]+max(M[i-1,j+1],M[i-1,j],M[i-1,j-1],M[i-1,j-2],M[i-1,j+2])
            else:
                M[i,j] = E[i,j]+max(M[i-1,j+1],M[i-1,j],M[i-1,j-1])
    line = M.shape[0] - 1
    # col = np.argmax(M[line,:])
    col = point
    # print(np.max(M[line,:]))
    path = [(0,0) for i in range(line+1)]

    path[line] = (line,point)

    direction = point
    
    for line in range(M.shape[0]-2,60,-1):
        if col == 0:
            if M[line,0] < M[line,1]:
                new_col = 1
            else:
                new_col = 0
        elif col == E.shape[1] - 1:
            if M[line,col] < M[line,col-1]:
                new_col = col - 1
            else:
                new_col = col
        elif col >= 2 and col <= E.shape[1] - 3:
            if not (M[line,col-1] == M[line,col] == M[line,col+1] == M[line,col-2] == M[line,col+2]):
                neigh = np.array([M[line,col-2],M[line,col-1],M[line,col],M[line,col+1],M[line,col+2]]) #TODO proiectia parcursului actual al masinii cu unghiul actual al rotilor
                new_col = col+np.argmax(neigh) - 2
            else:
                new_col = col - (direction - col)
                # new_col = col
                # if M[line,new_col] == 0 and new_col > 0:
                #     new_col -= 1
        else:
            if not (M[line,col-1] == M[line,col] == M[line,col+1]):
                neigh = np.array([M[line,col-1],M[line,col],M[line,col+1]]) #TODO proiectia parcursului actual al masinii cu unghiul actual al rotilor
                new_col = col+np.argmax(neigh) - 1
            else:
                new_col = col - (direction - col)
        # if col - 1 > 1 and col + 1 < E.shape[1] - 2:
        #     print(M[line,col-2],M[line,col-1],M[line,col],M[line,col+1],M[line,col+2],end=" ")
        #     print(line,end=" ")
        #     print(new_col)
        direction = col

        path[line]= (line,new_col)

        col = new_col

    return path

In [167]:
def create_path(img, pathL,pathR, color):
    
    new_image = img.copy()

    for row, col in pathL:
        new_image[row, col] = color

    for row, col in pathR:
        new_image[row, col] = color

    # E = compute_energy(img)
    # new_image_E = img.copy()
    # new_image_E[:,:,0] = E.copy()
    # new_image_E[:,:,1] = E.copy()
    # new_image_E[:,:,2] = E.copy()

    # for row, col in path:
    #     new_image_E[row, col] = color
    # cv2.imshow(name, np.uint8(new_image))
    return new_image
    # cv2.imshow('path E', np.uint8(new_image_E))
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


In [176]:
def preprocess(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    blurred = cv2.GaussianBlur(gray,(3,3),0)

    _, threshClassis = cv2.threshold(blurred, 190, 255, cv2.THRESH_BINARY)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_DILATE, (15, 15))
    # closing = cv2.morphologyEx(threshClassis, cv2.MORPH_CLOSE, kernel)

    return threshClassis


In [169]:
pre = preprocess(img)
preL = pre[:,:int(pre.shape[1]/2)]
preR = pre[:,int(pre.shape[1]/2):]
cv2.imshow('PreR', preR)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [170]:
pre = preprocess(img)
grad = compute_energy(pre)
cv2.imshow('Pre', pre)
cv2.imshow('grad', grad)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Histogram

In [171]:
def get_lane_start(img):
    histogram = np.sum(img[img.shape[0] // 2:, :], axis = 0)

    midpoint = int(histogram.shape[0] / 2)

    leftxBase = np.argmax(histogram[:midpoint])
    rightxBase = np.argmax(histogram[midpoint:]) + midpoint

    return leftxBase,rightxBase

In [172]:
pre = preprocess(img)
leftxBase,rightxBase = get_lane_start(pre)
# print(leftxBase)
# print(rightxBase)
pre[:20,:] = 0
# cv2.imshow("grad Image",pre)
grad = compute_energy_v2(pre)
# print(grad[104])
# cv2.imshow("grad1 Image",grad)
grad1 = compute_energy(pre)
# cv2.imshow("grad11 Image",grad1) 
# cv2.imshow("grad Image",pre)
# print(grad.shape)
pathL = select_dynamic_programming_path_going_up_point_start(grad,leftxBase)
pathR = select_dynamic_programming_path_going_up_point_start(grad,rightxBase)
print(pathL)
print(pathR)
new_img = create_path(img,pathL,pathR,RED) 
cv2.imshow("New Image",new_img)
cv2.imwrite("path.jpg",new_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (61, np.int64(10)), (62, np.int64(11)), (63, np.int64(13)), (64, np.int64(15)), (65, np.int64(17)), (66, np.int64(18)), (67, np.int64(20)), (68, np.int64(21)), (69, np.int64(22)), (70, np.int64(23)), (71, np.int64(25)), (72, np.int64(26)), (73, np.int64(27)), (74, np.int64(28)), (75, np.int64(29)), (76, np.int64(30)), (77, np.int64(30)), (78, np.int64(31)), (79, np.int64(32)), (80, np.int64(32)), (81, np.int64(33)), (82, np.int64(34)), (83, np.int64(34)), (84, np.int64(34)), (85, np.int64(35)), (86, np.int

### Pathfinding going always up

### Pathfinding going sideways as well

In [177]:
cap = cv2.VideoCapture('C:/Programs/facultate/licenta/my-autonomous-car/resources/1.mp4')
 
while cap.isOpened():
    ret, frame = cap.read()
 
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    pre = preprocess(frame)
    leftxBase,rightxBase = get_lane_start(pre)
    pre[:20,:] = 0
    cv2.imshow("grad Image",pre)
    grad = compute_energy_v2(pre)
    pathL = select_dynamic_programming_path_going_up_point_start(grad,leftxBase)
    pathR = select_dynamic_programming_path_going_up_point_start(grad,rightxBase)
    new_img = create_path(frame,pathL,pathR,RED) 
    cv2.imshow('newImg', new_img)
    
    if cv2.waitKey(1) == ord('q'):
        break
 
cap.release()
cv2.destroyAllWindows()